# Using Filter to detect spam Email

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import re
import Functions

In [2]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, roc_auc_score,make_scorer,precision_score,f1_score
from scipy.stats import ks_2samp

In [3]:
from wordcloud import WordCloud
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import warnings

### We import our preprocessed data

In [5]:
df = pd.read_csv("/Users/user/Documents/Spam_Detection/Data/new_data_clear.csv") 

In [6]:
df.head()

,sender,receiver,date,subject,body,label,urls,date_dt,year,month,day,weekday,hour,is_weekend
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1,2008-08-05 23:31:02+00:00,2008,8,5,1,23,False
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1,2008-08-05 23:31:03+00:00,2008,8,5,1,23,False
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1,2008-08-06 08:28:00+00:00,2008,8,6,2,8,False
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1,2008-08-05 23:31:20+00:00,2008,8,5,1,23,False
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1,2008-08-05 23:31:21+00:00,2008,8,5,1,23,False


In [7]:
N = 4000  # Le nombre total de lignes souhaité

Stratified_Sample = (
    df.groupby('label', group_keys=False)
      .apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df)))))
      .sample(frac=1)  # Pour mélanger les lignes du résultat
      .reset_index(drop=True)
)
df=Stratified_Sample

C:\Users\user\AppData\Local\Temp\ipykernel_22628\1717732568.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df)))))


In [8]:
df['body'] = df['body'].str.replace('\n', '', regex=False)

### We create our train and test dataframe

In [9]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

### Most common words among spam analysis

In [10]:
spam_emails = df_train[df_train["label"] == 1]["body"]

In [11]:
word_list = []
for email in spam_emails:
    word_list.extend(Functions.spacy_tokenizer(email))

In [12]:
word_counts = Counter(word_list)

In [13]:
spam_keywords = [word for word, count in word_counts.most_common(20)]

In [14]:
print(spam_keywords)

['news', 'cnn', 'network', 'setting', 'cable', 'go', 'time', 'receive', 'email', 'daily', 'lp', 'story', 'replica', 'lllp', 'aug', 'alert', 'man', 'suspect', 'result', 'watch']


In [15]:
print(word_list)

['cnn', 'alerts', 'custom', 'alert', 'alert', 'custom', 'alertdrive', 'girl', 'mad', 'crazy', 'cool', 'invention', 'fri', 'aug', 'storyyou', 'agree', 'receive', 'email', 'result', 'preference', 'setting', 'manage', 'setting', 'click', 'alter', 'alert', 'criterion', 'frequency', 'unsubscribe', 'receive', 'custom', 'email', 'alert', 'click', 'cable', 'news', 'network', 'cnn', 'center', 'atlanta', 'georgia', 'cable', 'news', 'network', 'time', 'warner', 'companyall', 'rights', 'reserve', 'view', 'privacy', 'policy', 'term', 'daily', 'video', 'story', 'aug', 'pm', 'edt', 'videos', 'mom', 'party', 'club', 'nancy', 'grace', 'new', 'photo', 'casey', 'anthony', 'party', 'nightclub', 'daughter', 'caylee', 'vanish', 'woman', 'computer', 'spies', 'tiger', 'attack', 'teen', 'zoo', 'mistress', 'testify', 'trial', 'man', 'call', 'sandwich', 'john', 'junior', 'gotti', 'arrest', 'passenger', 'slide', 'plane', 'plane', 'evacuates', 'passengers', 'pipeline', 'firm', 'helps', 'save', 'gas', 'gorillas', '

### Now we test if our filter works

In [16]:
y_pred = [Functions.is_spam(" ".join(Functions.spacy_tokenizer(email)),spam_keywords) for email in df_test['body']]
y_true = [email for email in df_test['label']]

In [17]:
y_pred

[1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


In [18]:
y_true

[1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,


In [19]:
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision=precision_score(y_true, y_pred)
f1=f1_score(y_true,y_pred)

print("Confusion Matrix :\n", conf_matrix)
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision : {precision:.2f}")
print(f"Recall : {recall:.2f}")
print(f"f1 : {f1:.2f}")

Confusion Matrix :
 [[156 191]
 [205 248]]
Accuracy : 0.51
Precision : 0.56
Recall : 0.55
f1 : 0.56


In [20]:
print(spam_emails)

3994    CNN Alerts: My Custom Alert Alert Name: My Cus...
423     >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...
2991    If you are looking for good chemists. You wont...
1221    Best replica watches from IWC at Replica Class...
506     Dear 494b76ec8d1ebe7dfac4daa8531d26b1Summer is...
                              ...                        
1638    You won't need to complain about it's small $i...
3772    A good marriage is like a casserole, only thos...
1130    Dear 36c58116c33527a56845d3325ede234dSummer is...
860     >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...
3507    Hello! I am bored this evening. I am nice girl...
Name: body, Length: 1804, dtype: object


Our accuracy is pretty bad, the f1 score is 0.56

### Now lets use an other type of filter

We now use an heuristic which is detailed in our Functions.py file

In [21]:
y_pred = [Functions.is_spam_heuristic(email,spam_keywords) for email in df_test['body']]
y_true = [email for email in df_test['label']]

In [22]:
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision=precision_score(y_true, y_pred)
f1=f1_score(y_true,y_pred)

print("Confusion Matrix :\n", conf_matrix)
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision : {precision:.2f}")
print(f"Recall : {recall:.2f}")
print(f"f1 : {f1:.2f}")

Confusion Matrix :
 [[  0 347]
 [  3 450]]
Accuracy : 0.56
Precision : 0.56
Recall : 0.99
f1 : 0.72


Good improvement